In [ ]:
#Bunch of installations
!pip install torch transformers datasets peft accelerate bitsandbytes trl safetensors ipywidgets huggingface_hub python-dotenv --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.6/336.6 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import login
#login('xxx')

In [ ]:
#Loading the Model and Tokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
device_map = {'':0}
import torch
#bitsandbytes parameters

#Activate 4-bit precision base model loading
use_4bit = True
#Compute dtype for 4 bit base models
bnb_4bit_compute_dtype = 'float16'
#Quantization type
bnb_4bit_quant_type = 'nf4'
#Activate double quantization??
use_double_nested_quant = False
# BitsAndBytesConfig int-4 config

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_use_double_quant=use_double_nested_quant,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype
)

model_name = "meta-llama/Llama-3.2-3B-Instruct"

# Load the pretrained model
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, use_cache = False, device_map=device_map)
model.config.pretraining_tp = 1

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
#Lora Configuration for PEFT
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1

from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, AutoPeftModelForCausalLM

# LoRA config based on QLoRA paper
peft_config = LoraConfig(
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout,
        r=lora_r,
        bias="none",
        task_type="CAUSAL_LM",
)

In [ ]:
#Loading the dataset
import pandas as pd

df = pd.read_parquet("hf://datasets/vivekdugale/llama2_chat_mental_health_convo_amod_3.51k/data/train-00000-of-00001.parquet")
df.head()

,text
0,<s>[INST] I'm going through some things with m...
1,<s>[INST] I'm going through some things with m...
2,<s>[INST] I'm going through some things with m...
3,<s>[INST] I'm going through some things with m...
4,<s>[INST] I'm going through some things with m...


In [ ]:
df['user_query'] = df['text'].apply(lambda x: x.split('[/INST]')[0].split('<s>[INST]')[1])
df['assistant_query'] = df['text'].apply(lambda x: x.split('[/INST]')[1].split('</s>')[0])

In [ ]:
#Test

'''
The dataset was created for Llama Based models. We need to convert it in the model that will fit the Qwen prompt.
'''
def format_text(user_query, assistant_query):
  messages = [
    {"role": "system",
     "content": '''You are a compassionate and knowledgeable mental health assistant.
     Your primary goal is to provide empathetic, non-judgmental, and evidence-based responses to
     users seeking help with mental health concerns. You must adhere to the following guidelines:

    1. **Domain Focus**: Only respond to questions or discussions related to mental health, emotional well-being, and psychological support.
    If a question falls outside this domain, politely redirect the user to consult a relevant professional or resource.

    2. **Empathy and Support**: Always respond with kindness, understanding, and validation.
    Acknowledge the user's feelings and experiences before providing any advice or information.

    3. **Safety First**: Avoid giving medical diagnoses, treatment plans, or specific medical advice.
    Instead, encourage users to seek help from licensed mental health professionals for personalized care.

    4. **Clarity and Conciseness**: Provide clear, concise, and actionable information.
    Use simple language to ensure your responses are accessible to everyone.

    5. **Evidence-Based**: Base your responses on widely accepted psychological principles and practices.
    Avoid speculative or unverified information.

    6. **Encourage Professional Help**: When appropriate, gently suggest that the
    user consult a licensed therapist, counselor, or healthcare provider for further assistance.

    7. **Crisis Situations**: If a user expresses thoughts of self-harm, suicide, or harm to others,
    immediately provide crisis resources (e.g., hotlines, emergency contacts) and encourage them to seek immediate professional help.

    Remember, your role is to support and guide users, not to replace professional mental health care.
    Always prioritize the user's well-being and safety.'''},
    {"role": "user", "content": user_query},
    {'role':'assistant', 'content': assistant_query}
  ]
  text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
  )
  return text


In [ ]:
df['text'] = df.apply(
    lambda row: format_text(row['user_query'], row['assistant_query']), axis=1)

In [ ]:
#Drop the assistant_query and user_querey columns
df = df.drop(['assistant_query', 'user_query'], axis=1)
#Change the dataset from pandas df to Dataset provided by hugging face
from datasets import Dataset

dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['text'],
    num_rows: 3512
})

In [ ]:
from transformers import TrainingArguments, Trainer
from trl import SFTTrainer

output_dir = 'Llama-3.2-finetuned-model'
#Number of training epochs
num_train_epochs = 1
#Enable fp16
fp16 = True
#Batch size per training
per_device_train_batch_size = 2
#Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 2
#Enable gradient checkpoint
gradient_checkpointing = True
#Gradient Clipping
max_grad_norm = 0.3
#Learning Rate
learning_rate = 2e-4
#Weight decay
weight_decay = 0.01
#Optimizer to use
optim = 'paged_adamw_32bit'
#Learning rate schedule
lr_scheduler_type = 'cosine'
# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03
# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = False
# Save checkpoint every X updates steps
save_steps = 0
# Log every X updates steps
logging_steps = 25
# Disable tqdm
disable_tqdm= False
# SFTTrainer parameters
# Maximum sequence length to use
max_seq_length = 1024 #None
# Pack multiple short examples in the same input sequence to increase efficiency
packing = True #False


# Define the training arguments
args = TrainingArguments(
    output_dir="./FineTuned-Llama-3.2-3B-Instruct-Model",
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size, # 6 if use_flash_attention else 4,
    gradient_accumulation_steps=gradient_accumulation_steps,
    gradient_checkpointing=gradient_checkpointing,
    optim=optim,
    logging_steps=logging_steps,
    save_strategy="epoch",
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    max_grad_norm=max_grad_norm,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    disable_tqdm=disable_tqdm,
    report_to="tensorboard",
    seed=42
)

In [ ]:
import transformers
from trl import SFTConfig, SFTTrainer
# Create the trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    #max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    #packing=packing,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
    args=args)
# train the model
trainer.train() # there will not be a progress bar since tqdm is disabled

# save model in local
trainer.save_model()

<ipython-input-11-a49065c7e639>:4: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Map:   0%|          | 0/3512 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
25,2.159600
50,1.363900
75,1.060100
100,1.055500
125,0.994100
150,1.017300
175,1.047300
200,1.050400
225,1.042700
250,1.067000


In [ ]:
from transformers import AutoModelForCausalLM

# Load the base model in FP16 (no quantization)
base_model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-3B-Instruct",
    torch_dtype="float16",  # Load in FP16
    device_map="auto"       # Use GPU if available
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from peft import PeftModel, PeftConfig
lora_weights_path = './FineTuned-Llama-3.2-3B-Instruct-Model'
# Load LoRA weights
lora_model = PeftModel.from_pretrained(base_model, lora_weights_path)

In [ ]:
# Merge LoRA weights with the base model
lora_model = lora_model.merge_and_unload()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from transformers import AutoTokenizer, TextStreamer, AutoModelForCausalLM
from peft import PeftModel
# Set up the TextStreamer
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

# Define the input prompt
input_prompt_1 = "I am very sad. What should i do?"

# Tokenize the input
input_prompt = format_text(input_prompt_1, '')
input_ids = tokenizer(input_prompt, return_tensors="pt").input_ids.to('cuda')

# Generate text with streaming
lora_model.generate(
    input_ids=input_ids,
    max_new_tokens=200,
    streamer=streamer,
    temperature=0.7,  # Adjust for creativity
    top_k=50,# Adjust for focused outputs
    eos_token_id=tokenizer.eos_token_id
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


　It is great that you are reaching out and wanting to do something about your feelings.  It is normal to feel sad at times, and there are many things that you can do to help yourself feel better.  You may want to talk to a friend, family member, or a professional counselor.  They can help you to identify what is causing your feelings of sadness and find ways to make you feel better.  You can also talk to a trusted adult about how you are feeling.  They can offer support and help you to find ways to make yourself feel better.  It is great that you are taking steps to address your feelings.  I hope that you find some things that make you feel better soon.  If you have any other questions, please don't hesitate to ask.  I wish you the best.  Best regards, Karen D.  Ph.D., L.P.C.  Phone: 973-


tensor([[128000, 128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,
           2696,     25,   6790,    220,   2366,     18,    198,  15724,   2696,
             25,    220,   1419,   4448,    220,   2366,     20,    271,   2675,
            527,    264,  60961,    323,  42066,  10723,   2890,  18328,     13,
            720,    257,   4718,   6156,   5915,    374,    311,   3493,  36681,
           5411,     11,   2536,  13636,    664,   9247,    278,     11,    323,
           6029,   6108,  14847,    311,    720,    257,   3932,  11125,   1520,
            449,  10723,   2890,  10742,     13,   1472,   2011,  49553,    311,
            279,   2768,  17959,   1473,    262,    220,     16,     13,   3146,
          13950,  26891,  96618,   8442,   6013,    311,   4860,    477,  20954,
           5552,    311,  10723,   2890,     11,  14604,   1664,  33851,     11,
            323,  24064,   1862,     13,    720,    262,   1442,    264,   3488,
          17503,   4994,    

In [ ]:
# lora_model.push_to_hub("Aspect05/Llama-3.2-3B-Instruct-Mental-Health-FP16", token = "xxx") # Online saving
# tokenizer.push_to_hub("Aspect05/Llama-3.2-3B-Instruct-Mental-Health-FP16", token = "xxx") # Online saving

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Aspect05/Llama-3.2-3B-Instruct-Mental-Health-FP16/commit/ca03625fc928364a825b3dedca4ab8a238b2db1c', commit_message='Upload tokenizer', commit_description='', oid='ca03625fc928364a825b3dedca4ab8a238b2db1c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Aspect05/Llama-3.2-3B-Instruct-Mental-Health-FP16', endpoint='https://huggingface.co', repo_type='model', repo_id='Aspect05/Llama-3.2-3B-Instruct-Mental-Health-FP16'), pr_revision=None, pr_num=None)

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Aspect05/Llama-3.2-3B-Instruct-Mental-Health-FP16")
model = AutoModelForCausalLM.from_pretrained("Aspect05/Llama-3.2-3B-Instruct-Mental-Health-FP16")

In [ ]:
# Set up the TextStreamer
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

# Define the input prompt
input_prompt = "Tell me about mental health in short"

# Tokenize the input
input_ids = tokenizer(input_prompt, return_tensors="pt").input_ids.to('cuda')

# Generate text with streaming
model.generate(
    input_ids=input_ids,
    max_new_tokens=500,
    streamer=streamer,
    temperature=0.7,  # Adjust for creativity
    top_k=50,         # Adjust for focused outputs
)